In [1]:
library(CellChat)
library(ggplot2)
library(ggalluvial)
library(svglite)
library(Seurat)
library(mindr)
options(stringsAsFactors = FALSE)
library(rjson)

# load data

In [2]:
dataobj <-readRDS('/stor/public/chenyx/HHCAd/unit_organized/RA.seuratobj.rds')

In [3]:
unique(dataobj$cell_type)

Loading required package: SeuratObject

Registered S3 methods overwritten by 'SeuratObject':
  method                     from  
  $.JackStrawData            Seurat
  $.Seurat                   Seurat
  $.SeuratCommand            Seurat
  $<-.Seurat                 Seurat
  [.Assay                    Seurat
  [.DimReduc                 Seurat
  [.Seurat                   Seurat
  [.SeuratCommand            Seurat
  [.SpatialImage             Seurat
  [[.Assay                   Seurat
  [[.DimReduc                Seurat
  [[.Seurat                  Seurat
  levels<-.Seurat            Seurat
  .DollarNames.JackStrawData Seurat
  .DollarNames.Seurat        Seurat
  .DollarNames.SeuratCommand Seurat
  as.list.SeuratCommand      Seurat
  as.logical.JackStrawData   Seurat
  dim.Assay                  Seurat
  dim.DimReduc               Seurat
  dim.Neighbor               Seurat
  dim.Seurat                 Seurat
  dim.SpatialImage           Seurat
  dimnames.Assay             Seurat
  dimna

[1] "Cardiomyocyte cell" "Fibroblast"         "Endothelial cell"  
[4] "Myelocyte"          "Pericyte"           "Smooth muscle cell"
[7] "Neuron"             "Lymphocyte"         "Adipocyte"

In [4]:
data.input <- dataobj@assays$RNA@data
identity <-data.frame(dataobj$cell_type)

# build cellchat object

In [5]:
# create a CellChat object
cellchat <- createCellChat(data = data.input)
# add Metadata
cellchat <- addMeta(cellchat, meta = identity, meta.name = "labels")
cellchat <- setIdent(cellchat, ident.use = "labels") 
groupSize <- as.numeric(table(cellchat@idents)) # number of cells in each cell group
CellChatDB <- CellChatDB.human 
cellchat@DB <- CellChatDB

In [ ]:
# Preprocessing
cellchat <- subsetData(cellchat)
cellchat <- identifyOverExpressedGenes(cellchat)
cellchat <- identifyOverExpressedInteractions(cellchat)
cellchat <- projectData(cellchat, PPI.human)
cellchat <- computeCommunProb(cellchat)
cellchat <- computeCommunProbPathway(cellchat)
cellchat <- aggregateNet(cellchat)
cellchat <- netAnalysis_signalingRole(cellchat, slot.name = "netP")

Issue identified!! Please check the official Gene Symbol of the following genes:  
 MGI:1915803 MGI:107675 MGI:1913728 MGI:107674 MGI:2157942 MGI:1347245 MGI:2135589 MGI:1918059 MGI:2136934 MGI:2136650 MGI:3522133 MGI:2664358 ITGAL_ITGB2L MGI:95957 MGI:95914 MGI:95915 MGI:1276570 MGI:95928 MGI:95929 MGI:95931 MGI:95936 H2-Q8 MGI:95956 MGI:95958 MGI:95959 H2-T9 MGI:1276527 MGI:2676637 MGI:95913 MGI:1276526 MGI:1276525 MGI:2687364 MGI:95942 MGI:95935 MGI:1276524 MGI:892004 MGI:95896 MGI:95904 MGI:1276522 H2-T18 H2-Q9 MGI:3704134 H2-L MGI:3702919 MGI:95933 MGI:2442805 MGI:3779381 MGI:95917 MGI:3647514 H2-BI H2-D MGI:109477 H60a H2-Ea-ps MGI:95895 MGI:103070 MGI:95901 MGI:95922 MGI:95923 MGI:107741 MGI:104849 MGI:1195970 MGI:1889024 MGI:3042847 MGI:1921853 MGI:3036289 MGI:2443630 MGI:1920500 MGI:107539 MGI:96877 MGI:2442965 MGI:1916392 ITGAM_ITGB2L MGI:2652397 MGI:96876 MGI:2159681 


In [ ]:
cellchat

In [ ]:
pathways <- cellchat@netP$pathways

In [ ]:
pathways

In [ ]:
saveRDS(cellchat,"/stor/public/chenyx/HHCAd/CellChat_result/RA.rds")

# build relationplot json

In [2]:
cellchat<- readRDS("/stor/public/chenyx/HHCAd/CellChat_result/RA.rds")

In [3]:
js_nodelist <- toJSON(data.frame(levels(cellchat@idents)))

In [4]:
js_nodelist

[1] "{\"levels.cellchat.idents.\":[\"Adipocyte\",\"Cardiomyocyte cell\",\"Endothelial cell\",\"Fibroblast\",\"Lymphocyte\",\"Myelocyte\",\"Neuron\",\"Pericyte\",\"Smooth muscle cell\"]}"

In [5]:
prop <- read.csv("/stor/public/liwenrui_heart/heart_cell_proportion/Right_atria.csv",row.names = 1)
prop <- prop/sum(prop)
prop <- round(prop,3)

In [6]:
js_nodes <- '['
for(i in 1:length(levels(cellchat@idents))){
    js_nodes <- paste0(js_nodes,'{"name":"',levels(cellchat@idents)[i],'","category":',i-1,',"label":{"show": true},"symbolSize":',as.integer(5+as.double(prop[gsub(" ",".",levels(cellchat@idents)[i])])*50),"}")# json index should start at 0
    if(i != length(levels(cellchat@idents))){
        js_nodes <- paste0(js_nodes,',')
    }else{
        js_nodes <- paste0(js_nodes,']')
    }
}

In [7]:
js_nodes

[1] "[{\"name\":\"Adipocyte\",\"category\":0,\"label\":{\"show\": true},\"symbolSize\":6},{\"name\":\"Cardiomyocyte cell\",\"category\":1,\"label\":{\"show\": true},\"symbolSize\":16},{\"name\":\"Endothelial cell\",\"category\":2,\"label\":{\"show\": true},\"symbolSize\":14},{\"name\":\"Fibroblast\",\"category\":3,\"label\":{\"show\": true},\"symbolSize\":16},{\"name\":\"Lymphocyte\",\"category\":4,\"label\":{\"show\": true},\"symbolSize\":8},{\"name\":\"Myelocyte\",\"category\":5,\"label\":{\"show\": true},\"symbolSize\":8},{\"name\":\"Neuron\",\"category\":6,\"label\":{\"show\": true},\"symbolSize\":6},{\"name\":\"Pericyte\",\"category\":7,\"label\":{\"show\": true},\"symbolSize\":9},{\"name\":\"Smooth muscle cell\",\"category\":8,\"label\":{\"show\": true},\"symbolSize\":7}]"

In [8]:
writeLines(js_nodelist, "/data8t/HHP/New_Website/Adult/Page2/RelationshipPlot/RA/nodelist.json") 
writeLines(js_nodes, "/data8t/HHP/New_Website/Adult/Page2/RelationshipPlot/RA/nodes.json") 

In [20]:
ps_list <- array(0,dim = c(length(levels(cellchat@idents)),length(levels(cellchat@idents)),length(pathways)))
for(p in 1:length(pathways)){
    pathway <- pathways[p]

    ############ From https://github.com/sqjin/CellChat/blob/master/R/visualization.R 188-223
    signaling <- pathway
    pairLR <- searchPair(signaling = pathway, pairLR.use = cellchat@LR$LRsig, key = "pathway_name", matching.exact = T, pair.only = T)
    signaling.name <- signaling
    net <- cellchat@net

    pairLR.use.name <- dimnames(net$prob)[[3]]
    pairLR.name <- intersect(rownames(pairLR), pairLR.use.name)
    pairLR <- pairLR[pairLR.name, ]
    prob <- net$prob
    pval <- net$pval

    prob[pval > 0.05] <- 0
    if (length(pairLR.name) > 1) {
    pairLR.name.use <- pairLR.name[apply(prob[,,pairLR.name], 3, sum) != 0]
    } else {
    pairLR.name.use <- pairLR.name[sum(prob[,,pairLR.name]) != 0]
    }


    if (length(pairLR.name.use) == 0) {
    stop(paste0('There is no significant communication of ', signaling.name))
    } else {
    pairLR <- pairLR[pairLR.name.use,]
    }
    nRow <- length(pairLR.name.use)

    prob <- prob[,,pairLR.name.use]
    pval <- pval[,,pairLR.name.use]

    if (length(dim(prob)) == 2) {
    prob <- replicate(1, prob, simplify="array")
    pval <- replicate(1, pval, simplify="array")
    }
    prob <-(prob-min(prob))/(max(prob)-min(prob))
    prob.sum <- apply(prob, c(1,2), sum)
    prob.sum <-(prob.sum-min(prob.sum))/(max(prob.sum)-min(prob.sum))*10 # 参数待定
    prob.sum <- signif(prob.sum,3)
    ############
    
    ps_list[,,p]<- prob.sum
    ############

    js_edge <-'['
    for(i in 1:length(levels(cellchat@idents))){
        for(j in 1:length(levels(cellchat@idents))){
            if(prob.sum[i,j]!=0){
                js_edge <- paste0(js_edge, '{"source":',i-1,',"target":',j-1,',"weight":',prob.sum[i,j],'},')# json index should start at 0
            }
        }
    }
    js_edge <- substr(js_edge,1,nchar(js_edge)-1)
    js_edge <- paste0(js_edge,']')
    writeLines(js_edge, paste0("/data8t/HHP/New_Website/Adult/Page2/RelationshipPlot/RA/",pathway,".json"))
}

# build heatmap json

In [21]:
levels(cellchat@idents)

[1] "Adipocyte"          "Cardiomyocyte cell" "Endothelial cell"  
[4] "Fibroblast"         "Lymphocyte"         "Myelocyte"         
[7] "Neuron"             "Pericyte"           "Smooth muscle cell"

In [22]:
js_vertical <- toJSON(pathways)
js_horizontal <-toJSON(data.frame(levels(cellchat@idents)))
writeLines(js_vertical, "/data8t/HHP/New_Website/Adult/Page2/HeatMap/RA/vertical.json") 
writeLines(js_horizontal, "/data8t/HHP/New_Website/Adult/Page2/HeatMap/RA/horizontal.json")

In [23]:
for( ct in 1:length(levels(cellchat@idents))){
    mat <- t(ps_list[ct,,])
    
    js_heatmap <- "["
    for(i in 1:dim(mat)[1]){
        for(j in 1:dim(mat)[2]){
            js_heatmap <- paste0(js_heatmap,'[',i-1,',',j-1,',"',mat[i,j],'"],') #防止0产生的灰色格子
        }
    }
    js_heatmap <- substr(js_heatmap,1,nchar(js_heatmap)-1)
    js_heatmap <- paste0(js_heatmap,']')
    
    writeLines(js_heatmap, paste0("/data8t/HHP/New_Website/Adult/Page2/HeatMap/RA/",levels(cellchat@idents)[ct],".json"))
}